# Simple land synthetic

An example of creating a simple land elastic synthetic. First a function for plotting.

In [ ]:
import numpy as np
def createPlot(data,p1,p2,p3):
    if isinstance(data,np.ndarray):
        fld=data
        hyper=Hypercube.hypercube(ns=[data.shape[2],data.shape[1],data.shape[0]])
    else:
        fld=data.getNdArray()
        hyper=data.getHyper()
    ax1=hyper.axes[0]
    ax2=hyper.axes[1]
    ax3=hyper.axes[2]
    mn=fld.min()
    mx=fld.max()
    front=fld[p3,:,::-1].T
    top=fld[::-1,:,p1]
    side=fld[:,p2,::-1].T
    b=(ax1.o,ax1.o+ax1.d*(ax1.n-1),ax2.o,ax2.o+ax2.d*(ax2.n-1))
    frontF=hv.Image(front,kdims=["X","Z"],bounds=(ax2.o,ax1.o,ax2.o+ax2.d*(ax2.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,invert_yaxis=True,aspect=1,axiswise=True)
    sideF=hv.Image(side,kdims=["Y","Z"],bounds=(ax3.o,ax1.o,ax3.o+ax3.d*(ax3.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,yaxis=None,invert_yaxis=True,aspect=1.,axiswise=True)
    topF=hv.Image(top,kdims=["X","Y"],bounds=(ax2.o,ax2.o,ax2.o+ax2.d*(ax2.n-1),ax3.o+ax3.d*(ax3.n-1))).opts(\
             cmap="gist_rainbow",clim=(mn,mx),toolbar=None,xaxis=None,aspect=1,axiswise=True)
    botF=hv.Layout(frontF+sideF).opts(toolbar=None)
    tot=hv.Layout(topF+hv.Empty()+frontF+sideF).cols(2).opts(toolbar=None)
    return tot
    
    

We will begin by installing a basement at 5500 m/s.

In [ ]:
import syntheticModel
import holoviews as hv
hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,800,800],os=[0,0,0],ds=[5,5,5])
basement=syntheticModel.geoModel(hyper,{"vp":3000.,"vs":2000,"rho":2000},"vp")

Lets define several different types of deposity functions.

In [ ]:
vsPars={"vs_dev":100,"vs_noiseZStretch":10}
rhoPars={"rho_dev":130,"rho_noiseZStretch":10}
depositC=syntheticModel.deposit(**vsPars,**rhoPars)  #Constant velocity
depositI=syntheticModel.deposit(interbedThick=20.,interbedThickDev=13.,interbedPropDev=100.,**vsPars,**rhoPars) #Interbeded layer
depositN=syntheticModel.deposit(noiseDev=300.,**vsPars,**rhoPars) #Add some noise to constant velocity\
depositIC=syntheticModel.deposit(interbedThick=15.,interbedThickDev=13.,interbedPropDev=100.,interbedDev=4.,**vsPars,**rhoPars) #Interbed with noise
depositICN=syntheticModel.deposit(noiseDev=120,interbedThick=15.,\
                                  interbedThickDev=13.,noiseFrequency=3,noiseZStretch=10,interbedPropDev=100.,
                                  interbedDev=4.,**vsPars,**rhoPars)

We are going to do one big deposition with a gradient function.

In [ ]:
update=depositICN.apply(basement,thick=400, prop=1500,seed=23,depthGrad=15,vs_prop=1500,rho_prop=300)
plt=createPlot(update.get(),40,400,400)
plt

In [ ]:
vs=update.getFloatField("vs").getNdArray()
de=update.getFloatField("rho").getNdArray()
print(vs.min(),vs.max(),de.min(),de.max(),"MIN MAX")

In [ ]:
plt=createPlot(update.getFloatField("rho"),40,400,400)
plt

In [ ]:
import syntheticModel.event as event
import syntheticModel.sincTable as sincTable
import syntheticModel.syntheticModel as syntheticModel
from syntheticModel.model_noise import fractal
from numba import jit, njit, prange, float64
import numpy as np
import datetime
import math

RAND_INT_LIMIT = 100000
FREQUENCY_LEVELS = [3, 6, 12, 24]
AMPLITUDE_LEVELS = [1, 0.5, 0.125, 0.0625]
#@njit(float64[:,:](float,float64[:,:],int,int),parallel=True)
@njit(parallel=True)
def rotateField(azimuth:float,field:np.ndarray,n1:int,n2:int):
    cIn=int(field.shape[0]/2.)
    cOut1=int(n1/2)
    cOut2=int(n2/2)
    rot=azimuth/180*math.pi
    out=np.zeros((n2,n1))
    for i2 in prange(n2):
        f2=(i2-cOut2)
        for i1 in range(n1):
            f1=(i1-cOut1)
            b1=math.cos(rot)*f1-f2*math.sin(rot)+cIn
            b2=math.cos(rot)*f2+f1*math.sin(rot)+cIn
            il1=int(b1)
            il2=int(b2)
            b1-=il1
            b2-=il2

            out[i2,i1]=(1.-b1)*(1.-b2)*field[il2,il1]+\
              (   b1)*(1.-b2)*field[il2,il1+1]+\
              (1.-b1)*(   b2)*field[il2+1,il1]+\
              (   b1)*(   b2)*field[il2+1,il1+1]

            #out[i2,i1]=field[il2,il1]
    return out

            
    


def getNoiseField(n1,n2,maxS,widthInLine,widthCrossLine,azimuth,noise_levels,begFreq):

    nmax=int(max(n1,n2)*1.8)
    # randomly shift origin
    o1 = np.random.rand() * nmax
    o2 = np.random.rand() * nmax
    p1 = np.linspace(o1, o1 + nmax*widthInLine, nmax, endpoint=False)
    p2 = np.linspace(o2, o2 + nmax*widthCrossLine , nmax, endpoint=False)
    p = np.stack(np.meshgrid(p1, p2, indexing='ij'), -1)
    p = np.reshape(p, (-1, 2)).T

    # get freq and amplitude values for perlin octaves
    freq=[begFreq]
    for i in range (3):
        freq.append(freq[-1]*2)

    freq_levels = list(np.array(freq[:noise_levels]) / (n1 ))
    amp_levels = AMPLITUDE_LEVELS[:noise_levels]
    field = fractal(p,
                    frequency=freq_levels,
                    n_octaves=noise_levels,
                    amplitude=amp_levels)
    field = np.reshape(field, (nmax, nmax))

    # normalize between -1 and 1
    mx=np.max(field)
    mn=np.min(field)
    field = (field+mn)/(mx-mn) *maxS
    m=rotateField(azimuth,field,n1,n2)
    return m


@njit(parallel=True)
def shiftFieldF(maxS:int,sincT:np.ndarray,outF:np.ndarray,inF:np.ndarray,shiftF:np.ndarray,\
               fill:float,basement:float):
    for i3 in prange(outF.shape[0]):
        for i2 in prange(outF.shape[1]):
            useF=maxS-shiftF[i3,i2]
            imin=math.ceil(useF)
            itab=min(sincT.shape[0]-1,int((imin-useF)*sincT.shape[0]+.5))
            
            for i1 in range(outF.shape[2]):
                #outF[i3,i2,i1]=inF[i3,i2,max]
                outF[i3,i2,i1]=0
                for isinc in range(sincT.shape[1]):
                    il=min(inF.shape[2]-1,max(0,i1-3+isinc-imin))
                    outF[i3,i2,i1]+=\
                    inF[i3,i2,il]*sincT[itab,isinc]
                if i3==490 and i2==709 and i1==200:
                    print(i1,imin,itab,outF[i3,i2,i1])


@njit(parallel=True)
def shiftFieldI(maxS:int,outF:np.ndarray,inF:np.ndarray,shiftF:np.ndarray,\
               fill:float,basement:float):
    for i3 in prange(outF.shape[0]):
        for i2 in prange(outF.shape[1]):
            imin=int(maxS-shiftF[i3,i2]+.5)
            for i1 in range(imin):
                outF[i3,i2,i1]=fill
            for i1 in range(inF.shape[2]):
                outF[i3,i2,i1+imin]=inF[i3,i2,i1]
            for i1 in range(inF.shape[2]+imin,outF.shape[2]):
                outF[i3,i2,i1]=basement            
class squish(event.event):
    """Default class for squishing"""
    def __init__(self,**kw):
        super().__init__(**kw)
    def applyBase(self,inM:syntheticModel.geoModel,azimuth:float=0.,\
        widthInLine:float=1000., widthCrossLine:float=100.,\
        maxShift:float=50, seed=None,begFreq:float=6.,nfreq=3)->syntheticModel.geoModel:
        """

        Squish a model

        Arguements:

        inM - Input model
        azimuth - Azimuth for squishing
        widthInLine - Approximate width for random patterns, axis 1
        widthCrosssLine - Approximate width for random patterns, axis 2
        maxShift -  Maximum shift in z
        seed - [null] int; random seed to initialize noise functions
        nfreq -3 Number of frequencies for noise (more means higher frequency bumpiness)
        begFreq - 3. Basic frequency level for noise (lower means lowe spatial frequency)

        Returns

            outModel - Returns updated model  
        """
        if seed is None:
            seed =int(datetime.datetime.utcnow().timestamp())
        np.random.seed(seed)

        axes=inM.getPrimaryProperty().getHyper().axes

        #calculate the maximum shift and then add cells to the top
        maxS=math.ceil(maxShift/axes[0].d)
        outM=inM.expand(maxS,0)

        sincT=sincTable.table

        shifts=getNoiseField(axes[1].n,axes[2].n,maxS,widthInLine/axes[1].n/axes[1].d,\
            widthCrossLine/axes[2].d/axes[2].n,azimuth, nfreq,begFreq)
        shifts-=shifts.min()
        self.shifts=shifts


        for fld in outM.getFloatFieldList():
            inp=inM.getFloatField(fld).getNdArray()
            base=inM.getBasementFloat(fld)
            outp=outM.getFloatField(fld).getNdArray()

            shiftFieldF(maxS,sincT,outp,\
                        inp,shifts,inM.getFillFloat(fld),base)

        for fld in outM.getIntFieldList():
            inp=inM.getIntField(fld).getNdArray()
            base=inM.getBasementInt(fld)
            
            shiftFieldI(maxS,outM.getIntField(fld).getNdArray(),\
                        inp,shifts,base,inM.getFillInt(fld))
        return outM




class basic(squish):
    def __init__(self,**kw):
        """
        Basic squish, for now we have not specialized
        
        """
        super().__init__(**kw)		

In [ ]:
squished2=basic().apply(update,begFreq=24,maxShift=470,seed=55)
#squished2=depositIC.apply(squished2,thick=70, prop=1450,seed=33)

plt=createPlot(squished2.get(),120,340,490)
plt

Lets deposit another layer

Now lets create a squish event

Lets create a bunch of faults.

In [ ]:
dep3=depositC.apply(squished2,thick=1, prop=1450,seed=58)
np.random.seed(99)
nb=1
ns=1
xs=np.random.uniform(low=.1,high=.9,size=(nb,ns))
ys=np.random.uniform(low=.1,high=.9,size=(nb,ns))
zs=np.random.uniform(low=.1,high=.9,size=(nb,ns))
angles=np.random.uniform(low=32,high=44,size=(nb,ns))
rads=np.random.uniform(low=1850,high=2150,size=(nb,ns))
shifts=np.random.uniform(low=130,high=160,size=(nb,ns))
lengths=np.random.uniform(low=300,high=440,size=(nb,ns))
azimuth=np.random.uniform(low=12,high=23,size=(nb,ns))
seeds=np.random.randint(99,size=(nb,ns))



In [ ]:
import syntheticModel.fault as fault
dep3=depositC.apply(squished2,thick=1, prop=1850,seed=86)

for i in range(xs.shape[0]):
    for j in range(xs.shape[1]):
        print("working on",j,i)
        dep3=syntheticModel.fault().apply(dep3,angle=angles[i,j],radius=rads[i,j],begx=xs[i,j],begy=ys[i,j],\
                            azimuth=azimuth[i,j],seed=seeds[i,j],\
        begz=zs[i,j],indicateI=True,shift=shifts[i,j],ruptureLength=lengths[i,j])
        print(f"beg={zs[i,j]},{ys[i,j]},{xs[i,j]} angle={angles[i,j]}, {lengths[i,j]}")
    #dep3=depositC.apply(dep3,thick=1,prop=1850)

    
    
plt=createPlot(dep3.get(),120,340,490)
plt

In [ ]:

s=squished2.get().getNdArray()
print(s.min(),s.max())
plt=createPlot(squished2.get(),200,340,490)
plt

n=s[490,709,200]
print(n)
